[View in Colaboratory](https://colab.research.google.com/github/nfmorenog/Sistemas-inteligentes/blob/master/canibal.ipynb)

Código para obtener `search.py` y `utils.py` del sitio de AIMA.

In [0]:
!wget https://raw.githubusercontent.com/aimacode/aima-python/master/search.py
!wget https://raw.githubusercontent.com/aimacode/aima-python/master/utils.py

--2018-09-03 22:33:30--  https://raw.githubusercontent.com/aimacode/aima-python/master/search.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55400 (54K) [text/plain]
Saving to: ‘search.py’

search.py           100%[===================>]  54.10K  --.-KB/s    in 0.03s   

2018-09-03 22:33:30 (2.05 MB/s) - ‘search.py’ saved [55400/55400]

--2018-09-03 22:33:31--  https://raw.githubusercontent.com/aimacode/aima-python/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21381 (21K) [text/plain]
Saving to: ‘utils.py’

utils.py    

In [0]:
!ls -la

total 108
drwxr-xr-x 1 root root  4096 Aug 27 20:57 .
drwxr-xr-x 1 root root  4096 Aug 27 20:53 ..
drwxr-xr-x 4 root root  4096 Aug 24 16:36 .config
drwxr-xr-x 2 root root  4096 Aug 27 20:57 __pycache__
drwxr-xr-x 2 root root  4096 Aug 24 16:46 sample_data
-rw-r--r-- 1 root root 55400 Aug 27 20:57 search.py
-rw-r--r-- 1 root root 21381 Aug 27 20:57 utils.py



## Introducción a los sistemas inteligentes 2018-1


___________

## El problema de la col, la cabra y el lobo (CCL).

Un granjero quiere transportar una col, una cabra y un lobo al otro lado del río.
Él tiene un bote que solo tiene espacio para transportar dos viajeros. El granjero no puede dejar la col y la cabra solos o la cabra y el lobo solos.

Su objetivo es modelar este problema como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________
## una clase para modelar el problema CCL

Un estado del problema puede representarse de diferentes formas. La sugerencia es usar una lista de cuatro posiciones (cada una correspondiente a un actor: granjero, col, cabra, lobo) que indique en que lado del rio (derecha o izquierda) se encuentra el respectivo actor. Por ejemplo el sigiente vector:

```python
['D', 'D', 'I', 'I']
```

Representa un estado en el cual el granjero y la col se encuentran en el lado derecho, y la cabra y el lobo en el izquierdo. Haría falta una posición más para representar dónde está el bote?

Asuma que al principio todos los actores están a la derecha y al final todos deben estar a la izquierda.

In [0]:
import search
import utils




class CCL_problem(search.Problem):    
    
    def __init__(self, startState):
        '''
        Store the initial state in the problem representation and any useful
        data.
        '''
        super().__init__(tuple(startState))
        self.expanded = 0
        pass
    
    
    def goal_test(self, state):
        '''
        Define when a given state is a goal state (all the actors at the left side of the river)
        '''
        if state == ('D',0,0,3,3):
          return True
        else:
          return False
    
    def getStartState(self):
        '''
        Implement a method that returns the start state.
        '''
        return self.initial
    
    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        self.expanded += 1
        #assert (state[action] == state[0])
        new_state = list(state)
        if state[0] == 'I':
            new_state[0] = 'D'
            new_state[1] -= action[0]
            new_state[2] -= action[1]
            new_state[3] += action[0]
            new_state[4] += action[1]
        else:
            new_state[0] = 'I'
            new_state[1] += action[0]
            new_state[2] += action[1]
            new_state[3] -= action[0]
            new_state[4] -= action[1]
        #print (new_state)
        return tuple(new_state)

    def actions(self, state):
        """Return the actions that can be executed in the given
        state."""
        
        actions = [(i,j) for i in range(3) for j in range(3)  if i + j  <=2 and i + j >0 and  self.valid_state(self.result( state, (i,j))) ]
        return actions
    
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1."""
        return c + 1
    
    def valid_state (self,state):
      if (state[1]==1 and state[2]>1) or (state[3]==1 and state[4]>1):        
        return False
      elif (state[1]==2 and state[2]>2) or (state[3]==2 and state[4]>2):
        return False
      elif (state[1]<0 or state[1]>3) or (state[2]<0 or state[2]>3) or (state[3]<0 or state[3]>3) or (state[4]<0 or state[4]>3):
        return False
      else:
        #print ("true")
        return True
    

## Búsqueda de una solución

Para encontrar una solución vamos a usar la función `breadth_first_tree_search` de la librería `search.py`

In [0]:
problem = CCL_problem(['I', 3, 3, 0, 0])
nodo = search.breadth_first_tree_search(problem)
nodo.path()

[<Node ('I', 3, 3, 0, 0)>,
 <Node ('D', 3, 1, 0, 2)>,
 <Node ('I', 3, 2, 0, 1)>,
 <Node ('D', 3, 0, 0, 3)>,
 <Node ('I', 3, 1, 0, 2)>,
 <Node ('D', 1, 1, 2, 2)>,
 <Node ('I', 2, 2, 1, 1)>,
 <Node ('D', 0, 2, 3, 1)>,
 <Node ('I', 0, 3, 3, 0)>,
 <Node ('D', 0, 1, 3, 2)>,
 <Node ('I', 0, 2, 3, 1)>,
 <Node ('D', 0, 0, 3, 3)>]

El método nos retorna un nodo que contiene el estado final. A partir de este podemos recuperar la secuencia de estados y acciones que me llevan desde el estado inicial hasta el estado objetivo.

In [0]:
tmp = nodo
while tmp != None:
  print(tmp, tmp.action)
  tmp = tmp.parent

<Node ('D', 0, 0, 3, 3)> (0, 2)
<Node ('I', 0, 2, 3, 1)> (0, 1)
<Node ('D', 0, 1, 3, 2)> (0, 2)
<Node ('I', 0, 3, 3, 0)> (0, 1)
<Node ('D', 0, 2, 3, 1)> (2, 0)
<Node ('I', 2, 2, 1, 1)> (1, 1)
<Node ('D', 1, 1, 2, 2)> (2, 0)
<Node ('I', 3, 1, 0, 2)> (0, 1)
<Node ('D', 3, 0, 0, 3)> (0, 2)
<Node ('I', 3, 2, 0, 1)> (0, 1)
<Node ('D', 3, 1, 0, 2)> (0, 2)
<Node ('I', 3, 3, 0, 0)> None


Esto puede hacerse de manera más sencilla con los métodos `path` y `solution`

In [0]:
nodo.path()

[<Node ('I', 3, 3, 0, 0)>,
 <Node ('D', 3, 1, 0, 2)>,
 <Node ('I', 3, 2, 0, 1)>,
 <Node ('D', 3, 0, 0, 3)>,
 <Node ('I', 3, 1, 0, 2)>,
 <Node ('D', 1, 1, 2, 2)>,
 <Node ('I', 2, 2, 1, 1)>,
 <Node ('D', 0, 2, 3, 1)>,
 <Node ('I', 0, 3, 3, 0)>,
 <Node ('D', 0, 1, 3, 2)>,
 <Node ('I', 0, 2, 3, 1)>,
 <Node ('D', 0, 0, 3, 3)>]

In [0]:
nodo.solution()

[(0, 2),
 (0, 1),
 (0, 2),
 (0, 1),
 (2, 0),
 (1, 1),
 (2, 0),
 (0, 1),
 (0, 2),
 (0, 1),
 (0, 2)]

In [0]:
nodo = search.depth_first_tree_search(problem)
nodo.path()

KeyboardInterrupt: ignored

In [0]:
nodo.depth

11